<a href="https://colab.research.google.com/github/AMAYadav/finance_models/blob/main/reft_ima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install functools --quiet
# # from functools import reduce
# # dfs = [df1, df2, df3, df4, df5, df6]
# # df_final = reduce(lambda left,right: pd.merge(left,right,on='some_common_column_name'), dfs)

In [ ]:
from pandas.core.internals.api import pandas_dtype
# from pandas.core.arrays.period import pandas_dtype
class reft_test:
  # try:
  import pandas as pd
  import numpy as np

  def __init__(self,price_security=None,start=None,end=None,option_security=None,time_to_maturity=None,):
    self.start=start
    self.end=end
    self.price_security=price_security #stock price
    self.option_security=option_security #option of stock
    # self.time_to_maturity=time_to_maturity
    self.start=start
    pass


  def annualized_time_to_maturity(self,start,end,):
    return np.busday_count(self.start,end,)/250

  def calc_apl(self,array):
    # apl= array-array.shift()
    apl=pd.DataFrame((array-array.shift()),index=array.index)
    apl.columns = ['apl']
    apl=apl.fillna(000)
    # apl= array-array.shift()
    # apl=pd.DataFrame(array-array.shift(),columns=['apl'])   # axis =0 > rows
    return apl


  def calc_hpl(self,stock,volatility,time,delta,gamma,vega,theta):

    hpl = pd.DataFrame(delta*(stock-stock.shift())+0.5*gamma*(stock-stock.shift())**2+vega*(volatility-volatility.shift())+theta*(time-time.shift()),columns=['hpl'],index=stock.index)
    hpl=hpl.fillna(000)
    return hpl

  def calc_rtpl(self,array,delta):
    array
    rtpl= pd.DataFrame((array-array.shift())*delta,columns=['rtpl'],index=array.index) # axis =0 > rows
    rtpl=rtpl.fillna(000)
    return rtpl


  def calc_delta_normal_var(self,delta:float=0.5,stock,option_value=1,alpha,t:float=1):
    # a=100-alpha
    stock=stock.fillna(000)
    stock=(stock*option_value*delta).stdev()
    t=t.astype(float)
    time=np.sqrt(1/t)
    var=(stock.quantile(alpha))*time
    # var=var.astype(float)
    # var=var

    #arr_float = np.array([1., 2., 3.], dtype=object)
    # arr_float64 = arr_float.astype(float)  # The solution proposed in other answers
    # np.exp(arr_float)  # This throws the TypeError
    # np.exp(arr_float64)
    var= pd.DataFrame(var,)
    var.columns=['var']
    return var

  def expected_shortfalls(self,delta:float=0.5,stock,option_value=1,alpha,t:float=1) ->float :

    stock=stock.fillna(000)
    stock=(stock*option_value*delta).stdev()
    t=t.astype(float)
    time=np.sqrt(1/t)
    var=(stock.quantile(alpha))*time
    # var=float(stock.quantile(alpha))
    percentage=var(alpha,stock)
    stock=stock[stock<=percentage]
    exp_shortfall=float(stock.mean())
    return exp_shortfall
    pass

  def exceptions(self,apl :pandas_dtype,hpl:pandas_dtype,rtpl:pandas_dtype,var:pandas_dtype):

    frame=pd.DataFrame()
    frame.exceptions_apl=0
    frame.exceptions_hpl=0


    frame=pd.merge(apl,hpl,left_index=True,right_index=True).merge(rtpl,left_index=True,right_index=True).merge(var,left_index=True,right_index=True)


    frame['exceptions_apl']= frame.loc[frame['apl']>frame['var']]=1
    frame['exceptions_apl']= frame.loc[frame['apl']<frame['var']]=0
    frame['exceptions_apl']

    frame['exceptions_hpl']= frame.loc[frame['hpl']>frame['var']]=1
    frame['exceptions_hpl']= frame.loc[frame['hpl']<frame['var']]=0
    frame['exceptions_hpl']

    results=dict()
    results['apl']=frame.exceptions_apl.sum()
    results['hpl']=frame.exceptions_hpl.sum()
    results['max_exceptions']=max(results['apl'],results['hpl'])


    return results
    pass

  def plat_test(self,hpl,rtpl,pnl):

    pass



# PLAT WORKING


In [ ]:
position='call'
k=1200
r=0.05


In [ ]:
import pandas as pd
expiry=pd.read_excel('/content/5819-PnL Attribution and Backtesting (PLAT).xlsm',sheet_name='data', parse_dates=True,)
data=pd.read_excel('/content/5819-PnL Attribution and Backtesting (PLAT).xlsm',sheet_name='data', parse_dates=True,).iloc[1:,]
# data_raw.iloc[1:,:].rename(columns=data_raw.iloc[1], inplace = False)
data.rename(columns=data.iloc[0],inplace=True)
data_new=data.iloc[1:]
# a=data_new[['Date','NIFTY_SPOT','Option']]

In [ ]:
data_new=data_new.loc[~(data_new==0).all(axis=1)]
data_new.info()
data_new.Date.astype('M8[D]')
data_new.set_index('Date',inplace=True)
# data_new.reset_index()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 222 entries, 2 to 223
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              222 non-null    object
 1   NIFTY_SPOT        222 non-null    object
 2   Option            222 non-null    object
 3   Time to Maturity  222 non-null    object
 4   BSM_vol           222 non-null    object
 5   d1                222 non-null    object
 6   d2                222 non-null    object
 7   BSM_value         222 non-null    object
 8   delta             222 non-null    object
 9   gamma             222 non-null    object
 10  vega              222 non-null    object
 11  theta             222 non-null    object
 12  APL ($)           222 non-null    object
 13  HPL ($)           222 non-null    object
 14  RTPL ($)          222 non-null    object
 15  1-day VaR (99%)   222 non-null    object
dtypes: object(16)
memory usage: 29.5+ KB


In [ ]:
# exp=pd.DataFrame(expiry.iloc[0,2:3].values,index=['expiry']).astype('M8[D]')
# exp.info()

In [ ]:
import numpy as np
import pandas as pd
maturity=reft_test()

In [ ]:
data_new.columns

Index(['NIFTY_SPOT', 'Option', 'Time to Maturity', 'BSM_vol', 'd1', 'd2',
       'BSM_value', 'delta', 'gamma', 'vega', 'theta', 'APL ($)', 'HPL ($)',
       'RTPL ($)', '1-day VaR (99%)'],
      dtype='object')

In [ ]:
hpl=maturity.calc_hpl(stock=data_new['NIFTY_SPOT'],volatility=data_new['BSM_vol'],time=data_new['Time to Maturity'], delta=data_new.delta, gamma=data_new.gamma, vega=data_new.vega,theta=data_new.theta)
hpl

,hpl
Date,
2020-02-07,0.000000
2020-02-10,-38.772673
2020-02-11,78.201714
2020-02-12,59.205852
2020-02-13,-26.852480
...,...
2020-12-23,137.196442
2020-12-24,150.547512
2020-12-28,128.746362


In [ ]:
rtpl=maturity.calc_rtpl(array=data_new['NIFTY_SPOT'],delta=data_new.delta)
rtpl

,rtpl
Date,
2020-02-07,0.000000
2020-02-10,-46.058657
2020-02-11,53.614425
2020-02-12,67.911792
2020-02-13,-19.245424
...,...
2020-12-23,134.799800
2020-12-24,148.150390
2020-12-28,123.950200


In [ ]:
var=maturity.calc_delta_normal_var(sigma=data_new['BSM_vol'],alpha=0.975,t=data_new['Time to Maturity'],delta=data_new.delta,stock=data_new['NIFTY_SPOT'])
data_new['var']=var
data_new.dropna(inplace=False)
data_new

,NIFTY_SPOT,Option,Time to Maturity,BSM_vol,d1,d2,BSM_value,delta,gamma,vega,theta,APL ($),HPL ($),RTPL ($),1-day VaR (99%),var
Date,,,,,,,,,,,,,,,,
2020-02-07,12098.34961,915,0.94,0.116048,0.546535,0.434023,915.0,0.707651,0.000252,4030.307058,-631.10089,0,0,0,0,14141.043348
2020-02-10,12031.5,869.95,0.936,0.117067,0.492987,0.379728,869.95,0.688989,0.000259,4112.378657,-628.152965,-45.05,-45.156212,-47.306185,-146.179305,14171.227039
2020-02-11,12107.90039,942,0.932,0.122323,0.529457,0.411366,942,0.701756,0.000243,4053.358586,-643.737833,72.05,72.497837,52.639041,-142.781825,14201.604838
2020-02-12,12201.2002,993.55,0.928,0.119166,0.606437,0.491641,993.55,0.727888,0.000237,3901.460415,-644.875151,51.55,51.157862,65.473693,-152.921769,14232.178835
2020-02-13,12174.65039,961.45,0.924,0.116542,0.5974,0.485375,961.45,0.72488,0.000245,3905.759684,-639.497294,-32.1,-32.061265,-19.325282,-155.713285,14262.951149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,13601.09961,1616.1,0.028,0.06,12.619064,12.609024,1617.887855,1,0.0,0.0,-599.160588,120.55,132.403637,134.7998,-198.131229,81934.416401
2020-12-24,13749.25,1764.4,0.024,0.05,17.726338,17.718593,1763.641363,1,0.0,0.0,-599.280432,148.3,145.753748,148.15039,-120.068732,88499.284495
2020-12-28,13873.2002,1896.4,0.016,0.05,23.064434,23.058109,1882.796361,1,0.0,0.0,-599.520192,132,119.155957,123.9502,-101.147154,108389.044807


In [ ]:
apl=maturity.calc_apl(data_new.NIFTY_SPOT)

In [ ]:
total_exceptions=maturity.exceptions(apl=apl,hpl=hpl,rtpl=rtpl,var=var)

print(f'Total_exceptions:{total_exceptions}')

Total_exceptions:{'apl': 0, 'hpl': 0, 'max_exceptions': 0}


In [ ]:
maturity.exceptions(apl=apl,hpl=hpl,rtpl=rtpl,var=var).items()

dict_items([('apl', 0), ('hpl', 0), ('max_exceptions', 0)])

In [ ]:
print(maturity.exceptions(apl=apl,hpl=hpl,rtpl=rtpl,var=var))

{'apl': 0, 'hpl': 0, 'max_exceptions': 0}
